In [1]:
import numpy as np
import random
import os
from google.colab import files
import shutil

In [2]:
def generate_vrp_instance(num_clients, max_x, max_y, max_demand, vehicle_capacity):
    """
    Gera uma instância do VRP.

    Parâmetros:
        - num_clients: Número de clientes.
        - max_x, max_y: Limites do plano para coordenadas.
        - max_demand: Demanda máxima de cada cliente.
        - vehicle_capacity: Capacidade do veículo.

    Retorna:
        - Um dicionário com a instância gerada.
    """
    # Coordenadas do depósito (0, 0)
    depot = (0, 0)

    # Gerar coordenadas aleatórias para os clientes
    coordinates = [depot]
    for _ in range(num_clients):
        x = random.randint(0, max_x)
        y = random.randint(0, max_y)
        coordinates.append((x, y))

    # Gerar demandas aleatórias para os clientes
    demands = [0]  # Depósito não tem demanda
    for _ in range(num_clients):
        demands.append(random.randint(1, max_demand))

    # Calcular matriz de distâncias
    def calculate_distance(p1, p2):
        return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

    num_nodes = num_clients + 1  # Clientes + depósito
    distance_matrix = [
        [calculate_distance(coordinates[i], coordinates[j]) for j in range(num_nodes)]
        for i in range(num_nodes)
    ]

    # Criar a instância
    instance = {
        "num_clients": num_clients,
        "coordinates": coordinates,
        "demands": demands,
        "vehicle_capacity": vehicle_capacity,
        "distance_matrix": distance_matrix
    }
    return instance


In [3]:
def save_instance(instance, filename):
    """
    Salva a instância do VRP em um arquivo de texto.
    """
    with open(filename, "w") as file:
        num_nodes = instance["num_clients"] + 1
        file.write(f"{num_nodes}\n")

        # Escrever coordenadas e demandas
        for i, (x, y) in enumerate(instance["coordinates"]):
            demand = instance["demands"][i]
            file.write(f"{i} {x} {y} {demand}\n")

        # Escrever capacidade do veículo
        file.write(f"{instance['vehicle_capacity']}\n")

        # Escrever matriz de distâncias
        for row in instance["distance_matrix"]:
            file.write(" ".join(map(str, row)) + "\n")


In [4]:
def generate_multiple_instances(num_instances, output_dir):
    """
    Gera várias instâncias e as salva como arquivos de texto.
    """
    for i in range(num_instances):
        # Parâmetros para cada instância
        num_clients = random.randint(5, 15)  # Número aleatório de clientes
        max_x = 100
        max_y = 100
        max_demand = 10
        vehicle_capacity = random.randint(20, 50)

        # Gerar instância
        instance = generate_vrp_instance(num_clients, max_x, max_y, max_demand, vehicle_capacity)

        # Salvar instância
        filename = f"{output_dir}/instance_{i + 1}.txt"
        save_instance(instance, filename)


In [5]:
# Criar instâncias

output_dir = "instances"
os.makedirs(output_dir, exist_ok=True)

# Gerar 10 instâncias e salvar
generate_multiple_instances(10, output_dir)

# Verificar arquivos gerados
import os
print(os.listdir(output_dir))


['instance_10.txt', 'instance_2.txt', 'instance_1.txt', 'instance_7.txt', 'instance_4.txt', 'instance_8.txt', 'instance_6.txt', 'instance_3.txt', 'instance_9.txt', 'instance_5.txt']


In [6]:
# Compactar instâncias em um arquivo ZIP
shutil.make_archive("vrp_instances", "zip", output_dir)

# Baixar o arquivo ZIP
files.download("vrp_instances.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
import ast

def process_csv(file_path):
    # Carregar o arquivo CSV
    df = pd.read_csv(file_path)
    
    # Criar lista para armazenar os dados transformados
    data_cleaned = []
    
    # Iterar sobre as linhas do DataFrame
    for _, row in df.iterrows():
        conj_cliente = row["conjunto"]
        media_dict = ast.literal_eval(row["media"])  # Converter string para dicionário
        
        for instancia, custo in media_dict.items():
            custo = custo.strip()
            if custo.isdigit():  # Apenas incluir valores numéricos válidos
                instancia = instancia.replace(".vrp", "")
                data_cleaned.append([conj_cliente, instancia, int(custo)])
    
    # Criar novo DataFrame com os dados limpos
    df_cleaned = pd.DataFrame(data_cleaned, columns=["Conj_cliente", "Instancia", "Custo"])
    
    return df_cleaned

# Exemplo de uso:
file_path = "/Users/jvmaues/Documents/Estudos/Mestrado/optimization/cvrp-cec/dados_media_j1.csv"  # Substitua pelo caminho correto
df_result = process_csv(file_path)
df_result


,Conj_cliente,Instancia,Custo
0,K7E4C18J1,X-n289-k60,97691
1,K7E4C18J1,X-n979-k58,123920
2,K7E4C18J1,CMT2,846
3,K7E4C18J1,Golden_4,14208
4,K7E4C18J1,X-n655-k131,107601
...,...,...,...
618,K5E5C15J1,X-n480-k70,92103
619,K5E5C15J1,CMT1,521
620,K5E5C15J1,CMT2,843
621,K5E5C15J1,X-n336-k84,144250


In [2]:
import pandas as pd


# Calculando a média do Custo por conj_clientes
df_result = df_result.groupby('Conj_cliente')['Custo'].mean().reset_index()

#df_result.to_csv('media_solucao_local_custo_j1.csv')

In [3]:
df_result

,Conj_cliente,Custo
0,K3E1C10J1,61196.555556
1,K3E1C15J1,49928.466667
2,K3E1C18J1,32420.812500
3,K3E2C10J1,28346.500000
4,K3E2C15J1,49987.571429
5,K3E2C18J1,44890.500000
6,K3E3C10J1,74885.750000
7,K3E3C15J1,53904.285714
8,K3E3C18J1,73599.388889
9,K5E1C10J1,36891.777778


In [4]:
df_result = df_result.rename(columns={'Custo': 'custo_solucao_local', 'Conj_cliente' : 'conj_cliente'})
df_result

,conj_cliente,custo_solucao_local
0,K3E1C10J1,61196.555556
1,K3E1C15J1,49928.466667
2,K3E1C18J1,32420.812500
3,K3E2C10J1,28346.500000
4,K3E2C15J1,49987.571429
5,K3E2C18J1,44890.500000
6,K3E3C10J1,74885.750000
7,K3E3C15J1,53904.285714
8,K3E3C18J1,73599.388889
9,K5E1C10J1,36891.777778


In [5]:
df_result.to_csv('custo_solucao_local.csv')